In [1]:
# ============================================================
# compute_pki_and_selectivity.py
# Predict pKi values for a common ligand set across all receptors and
# compute pairwise selectivity (ΔpKi)
# ============================================================

import glob
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import combinations
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors, Crippen, rdMolDescriptors

# ----------  Configuration ----------------------
SMILES_PATH = Path(
    "/Users/anastazja/magisterka 2024:2025/sem 1/phyton/Project/5HT5A/ML_LAB/5HT/ligands_for_inference.csv"
)
MODELS_DIR = Path("models/pKi")
OUTPUT_DIR = Path("data")
OUTPUT_DIR.mkdir(exist_ok=True)

FP_RADIUS = 2
N_BITS = 1024

# ---------- Descriptor function ----------------------
def physchem(mol: Chem.Mol) -> np.ndarray:
    """Compute 26 RDKit physicochemical descriptors."""
    return np.array([
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Crippen.MolMR(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.RingCount(mol),
        Descriptors.FractionCSP3(mol),
        rdMolDescriptors.CalcTPSA(mol),
        rdMolDescriptors.CalcNumAliphaticRings(mol),
        rdMolDescriptors.CalcNumAromaticRings(mol),
        rdMolDescriptors.CalcNumSaturatedRings(mol),
        rdMolDescriptors.CalcNumAliphaticCarbocycles(mol),
        rdMolDescriptors.CalcNumAromaticCarbocycles(mol),
        rdMolDescriptors.CalcNumSaturatedCarbocycles(mol),
        rdMolDescriptors.CalcNumAliphaticHeterocycles(mol),
        rdMolDescriptors.CalcNumAromaticHeterocycles(mol),
        rdMolDescriptors.CalcNumSaturatedHeterocycles(mol),
        rdMolDescriptors.CalcNumHeteroatoms(mol),
        rdMolDescriptors.CalcNumAmideBonds(mol),
        float(Chem.GetFormalCharge(mol)),
        rdMolDescriptors.CalcExactMolWt(mol),
        rdMolDescriptors.CalcNumAtomStereoCenters(mol),
        rdMolDescriptors.CalcNumUnspecifiedAtomStereoCenters(mol),
        Descriptors.NumValenceElectrons(mol),
        rdMolDescriptors.CalcLabuteASA(mol)
    ], dtype=np.float32)

# ---------- Featurization ----------------------
def featurize(smiles, radius: int = FP_RADIUS, nbits: int = N_BITS) -> np.ndarray:
    """Generate ECFP bit vectors and physchem descriptors for a list of SMILES."""
    fps, descs = [], []
    for smi in smiles:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            raise ValueError(f"Invalid SMILES: {smi}")
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nbits)
        arr_fp = np.zeros(nbits, dtype=np.uint8)
        DataStructs.ConvertToNumpyArray(fp, arr_fp)
        fps.append(arr_fp)
        descs.append(physchem(mol))
    X_fp = np.vstack(fps)
    X_desc = np.vstack(descs)
    return np.hstack([X_fp, X_desc])

# ---------- Main execution ----------------------
if __name__ == "__main__":
    # Load SMILES and compute features
    smiles = pd.read_csv(SMILES_PATH)["Smiles"]
    X = featurize(smiles)

    # Predict pKi for each receptor
    pki_predictions = {}
    for model_file in MODELS_DIR.glob("LGBM_pKi_*.pkl"):
        receptor = model_file.stem.split("_")[-1]
        model = joblib.load(model_file)
        scaler = joblib.load(MODELS_DIR / f"scaler_pKi_{receptor}.pkl")
        n_desc = scaler.mean_.shape[0]

        X_scaled = X.copy()
        X_scaled[:, -n_desc:] = scaler.transform(X_scaled[:, -n_desc:])
        pki_predictions[receptor] = model.predict(X_scaled)

    # Assemble pKi matrix
    pki_df = pd.DataFrame(pki_predictions)
    pki_df.insert(0, "Smiles", smiles)
    pki_out = OUTPUT_DIR / "pKi_matrix_pred.csv"
    pki_df.to_csv(pki_out, index=False)
    print(f"Saved pKi matrix to {pki_out} ({pki_df.shape[0]}×{pki_df.shape[1]})")

    # Compute selectivity ΔpKi for each receptor pair
    sel_df = pki_df.copy()
    receptors = list(pki_predictions.keys())
    for r1, r2 in combinations(receptors, 2):
        sel_df[f"Sel_{r1}_vs_{r2}"] = sel_df[r1] - sel_df[r2]

    sel_out = OUTPUT_DIR / "sel_matrix_pred.csv"
    sel_df.to_csv(sel_out, index=False)
    print(f"Saved selectivity matrix to {sel_out} ({sel_df.shape[0]}×{sel_df.shape[1]})")


Saved pKi matrix to data/pKi_matrix_pred.csv (49×1)
Saved selectivity matrix to data/sel_matrix_pred.csv (49×1)
